### Background info

Followed the 3-year reliability paper: https://doi.org/10.1016/j.neuroimage.2021.118516
1. "Using these source estimates, we then estimated the power of cortical activity in the canonical frequency bands: delta (2–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (15–30 Hz), low gamma (30–80 Hz), and high gamma (80–150 Hz). We used Welch's method for estimating power spectrum densities (PSD) per four-second epoch across each MEG recording, with 1-second sliding Hamming windows overlapping at 50%. We then standardized the PSD values at each frequency bin to the total power across the frequency spectrum. We then averaged PSD maps (ie. source estimates) across epochs for each participant to obtain one set of six PSD maps (one per frequency band) per participant per visit."
2. "We projected these maps onto the MNI ICBM152 brain template (Fonov et al., 2009) and applied a 3 mm FWHM smoothing kernel."
3. "Used single rater two-way mixed-effects model and absolute agreement definition, or ICC(A,1)... ICC estimates and their 95% confidence intervals were calculated using the Matlab Central file-exchange ICC.m function. This ICC calculation was applied at every vertex in the PSD maps to obtain spatially specific reliability estimates at each of the frequency bands. This resulted in an ICC map per frequency band."
4. "To further visualize the reliability of source power in each frequency band, regions of interest (Brainstorm “scouts”) in the frontal, parietal, temporal, and occipital lobes were applied to each participant's PSD map. The average power (relative to total spectral power) across each lobe was extracted for each participant and each visit. ICCs of these values were then calculated using the same ICC(A,1) model."
- "ICC .5 indicates poor reliability, values between .5 to .75 indicates moderate reliability, values between .75 and .9 indicates good reliability, and values greater than .9 indicate excellent reliability. Importantly, we evaluated the level of reliability based on the 95% confidence interval of the ICC estimate, not the estimate itself, since the interval reveals the chance that the true ICC value lands on any point between the bounds."

### Morph

Morph before sprint
- The 3yr reliability paper applied a 3mm smoothing kernel
Sources: 
- https://mne.tools/stable/auto_examples/inverse/morph_surface_stc.html

### ICC

https://github.com/raphaelvallat/pingouin/blob/dcfdc82bbc7f1ba5991b80717a5ca156617443e8/pingouin/reliability.py#L158
ICC2 (single random raters)

### Set up

In [ ]:
# Packages
import numpy as np
import pandas as pd
import pingouin as pg
import os
import pickle
from functools import partial
from multiprocessing import Pool

# Directory containing _fooof csv files
base_dir = "/home/isw3/scratch/sprint/output_noSSP_nomorph_winL4"

# Constants
N_WINDOWS = 115 # 0 to 114
N_VERTICES = 8196
N_ITERATIONS = 100






# WITHIN-SCAN

# Filenames: Day 1, rest 1
runnames = ["sub_NVAR008_251016_rest1", 
            "sub_NVAR010_251027_rest1"]

r = 2

# Pre-generate list of random windows
windowAs = []
for i in range(N_ITERATIONS):
    np.random.seed(i)
    windowA = np.random.choice(range(N_WINDOWS - r))
    windowAs.append(windowA)


# Pre-load all exponents
# data_cache is a dictionary; format data_cache[runname][vertex] = series of exponents
print("Pre-loading files:")
data_cache = {}
for runname in runnames:
    data_cache[runname] = {}
    for vertex in range(N_VERTICES):
        file_path = os.path.join(base_dir, runname + "_fooof_vertex" + str(vertex) + ".csv")
        df_file = pd.read_csv(file_path, usecols=["exponent"]) 
        data_cache[runname][vertex] = df_file["exponent"].values 
print("Done loading files.")


def compute_icc_for_vertex(vertex, windowA): 
    """ 
    Computes ICC on a single vertex
    Parameters: idx of one vertex; idx of one window pair
    Returns: [vertex idx, icc value] -> will be one row 
    """
    print(f"Now working on vertex {vertex} windowA {windowA}")

    try: 
        # Create dataframe of ratings
        ratings = []
        for runname in runnames: # Here, levels are runnames because there is only one per subject, but caution that this won't be true later
            ratings.extend([
                (runname, "valA", data_cache[runname][vertex][windowA]),
                (runname, "valB", data_cache[runname][vertex][windowA + r])
            ])
        df_vertex = pd.DataFrame(ratings, columns=["TARGET", "RATER", "RATING"])

        # Compute ICC
        try: 
            results = pg.intraclass_corr(data=df_vertex, targets='TARGET', raters='RATER', ratings='RATING')
        except Exception as e:
            print(f"pg.intraclass_corr for {vertex} {windowA} returned error: {e}")
        icc = results.loc[results['Type'] == 'ICC2', 'ICC'].values[0]
        icc = windowA

        # Return vertex and corresponding ICC
        return [vertex, icc]
    
    except Exception as e: 
        print(f"compute_icc_for_vertex for {vertex} {windowA} returned error: {e}")
        return None


def compute_icc():
    rows = []
    with Pool() as pool:
        for windowA in windowAs: 
            for result in pool.starmap(compute_icc_for_vertex, [(vertex, windowA) for vertex in range(N_VERTICES)]):
                if result is not None:
                    rows.append([windowA] + result)

    df = pd.DataFrame(rows, columns=["WINDOW_A", "VERTEX", "ICC"])
    
    # Fisher transform; won't work for values of exactly -1 or 1 so clip first
    df["ICC_Z"] = np.arctanh(df["ICC"].clip(-0.9999, 0.9999))

    with open(os.path.join(base_dir, "ICC_" + str(r) + ".pkl"), "wb") as f:
        pickle.dump(df, f)

    # Average per vertex
    df_avg = df.groupby("VERTEX")["ICC_Z"].mean().reset_index()
    with open(os.path.join(base_dir, "ICC_perVertex_" + str(r) + ".pkl"), "wb") as f:
        pickle.dump(df_avg, f)



compute_icc()

Pre-loading files:
Done loading files.


In [ ]:
# FOR LATER
# BETWEEN-SCANS: 

# Runames: To start, always use visit_1_scan_1 as the reference
comparison_dictionary = {
    "within-run" : {
        "sub_NVAR008" : ["sub_NVAR008_251016_rest1", "sub_NVAR008_251016_rest1"], 
        "sub_NVAR010" : ["sub_NVAR010_251027_rest1", "sub_NVAR010_251027_rest1"], 
        "sub_NVAR999" : ["sub_NVAR999_251016_rest1", "sub_NVAR999_251016_rest1"]
    },
    "day" : {
        "sub_NVAR008" : ["sub_NVAR008_251016_rest1", "sub_NVAR008_251017_rest1"], 
        "sub_NVAR010" : ["sub_NVAR010_251027_rest1", "sub_NVAR010_251028_rest1"], 
        "sub_NVAR999" : ["sub_NVAR999_251016_rest1", "sub_NVAR999_251017_rest1"]
    },
    "week" : {
        "sub_NVAR008" : ["sub_NVAR008_251016_rest1", "sub_NVAR008_251023_rest1"], 
        "sub_NVAR010" : ["sub_NVAR010_251027_rest1", "sub_NVAR010_251103_rest1"], 
        "sub_NVAR999" : ["sub_NVAR999_251016_rest1", "sub_NVAR999_251023_rest1"]
    },
    "month" : {
        "sub_NVAR008" : ["sub_NVAR008_251016_rest1", "sub_NVAR008_251113_rest1"], 
        "sub_NVAR010" : ["sub_NVAR010_251027_rest1", "sub_NVAR010_251124_rest1"], 
        "sub_NVAR999" : ["sub_NVAR999_251016_rest1", "sub_NVAR999_251113_rest1"]
    }
}



print("Pre-loading files:")
data_cache = {}
scale = comparison_dictionary["within-run"]
for subject in scale.keys(): 
    runnameA = scale[subject][0]
    runnameB = scale[subject][1]


for runname in runnames:
    data_cache[runname] = {}
    for vertex in range(N_VERTICES):
        file_path = os.path.join(base_dir, runname + "_fooof_vertex" + str(vertex) + ".csv")
        df_file = pd.read_csv(file_path, usecols=["exponent"]) 
        data_cache[runname][vertex] = df_file["exponent"].values 
print("Done loading files.")



ratings = []
scale = comparison_dictionary["within-run"]
for subject in scale.keys(): 
    runnameA = scale[subject][0]
    runnameB = scale[subject][1]

    ratings.extend([
            (subject, "valA", data_cache[runnameA][vertex][windowA]),
            (subject, "valB", data_cache[runnameB][vertex][windowA + r])
        ])
    df_vertex = pd.DataFrame(ratings, columns=["TARGET", "RATER", "RATING"])



 #   "40min" : {
 #       "sub_NVAR008" : ["sub_NVAR008_251016_rest1", "sub_NVAR008_251016_rest2"], 
 #       "sub_NVAR010" : ["sub_NVAR010_251027_rest1", "sub_NVAR010_251027_rest2"], 
 #       "sub_NVAR999" : ["sub_NVAR999_251016_rest1", "sub_NVAR999_251016_rest2"]
 #   },

Pre-loading files:


NameError: name 'runnames' is not defined

In [19]:
rows = []

for i in [0, 1, 2, 3]: 
    rows.append([i, i+2] + [i])

rows

[[0, 2, 0], [1, 3, 1], [2, 4, 2], [3, 5, 3]]